In [4]:
"""The pipes are arranged in a two-dimensional grid of tiles:

| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.

S is the starting position; there is a pipe on this tile,
but the sketch doesn't show what shape the pipe has.
"""

example_1_1 =""".....
.F-7.
.|.|.
.L-J.
.....
"""

example_1_2 =""".....
.S-7.
.|.|.
.L-J.
.....
"""

example_1_3 ="""-L|F7
7S-7|
L|7||
-L-J|
L|-JF
"""

example_2_1 ="""..F7.
.FJ|.
SJ.L7
|F--J
LJ...
"""

example_2_2 ="""7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
"""

example_3="""...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
"""

example_4=""".F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
"""

In [8]:
import numpy as np
from collections import deque
from tqdm import tqdm

starting_pos = (0,0)
# Define the possible pipe types
pipe_types = {
    '|': [(-1, 0), (1, 0)],
    '-': [(0, -1), (0, 1)],
    'L': [(0, -1), (1, 0)],
    'J': [(0, 1), (1, 0)],
    '7': [(0, 1), (-1, 0)],
    'F': [(0, -1), (-1, 0)],
    '.': [],
    'S': [(0, 1), (0, -1), (1, 0), (-1, 0)]
}

def split_input(input_data):
    return [list(row) for row in input_data.splitlines()]

def verify_symbol(mirrored_connections, local_grid_array, x_row, x_col):
    """Verify that the connections are valid for the given pipe type"""
    # Check each position relative to 'X'
    for dr, dc in mirrored_connections:
        new_row, new_col = x_row + dr, x_col + dc
        if 0 <= new_row < local_grid_array.shape[0] and 0 <= new_col < local_grid_array.shape[1]:
            if (dr,dc) not in pipe_types[local_grid_array[new_row, new_col][0]]:
                return None
        else:
            return None
    return mirrored_connections

def check_pipe(x_row, x_col, local_grid_array, original_char, debug=False):
    """
    Determine if a pipe is valid based on a 3x3 grid.
    Valid pipes have exactly two connections and is the expected
    character, or 'S'
    """
    # Check each position relative to 'X'
    valid_pipes = 0
    is_starting_pos = False
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_row, new_col = x_row + dr, x_col + dc
        if 0 <= new_row < local_grid_array.shape[0] and 0 <= new_col < local_grid_array.shape[1]:
            if debug:
                print(f"New row: {dr}, new col: {dc}, char: {local_grid_array[new_row, new_col]}")
            if (dr,dc) in pipe_types[local_grid_array[new_row, new_col][0]]:
                if debug:
                    print(f"Found valid pipe {local_grid_array[new_row, new_col]}")
                valid_pipes += 1
    # If there's exactly two valid pipes next to 'X', return True, else False
    if valid_pipes >= 2:
        if original_char != 'S':
            mirrored_connections = [(-dr, -dc) for dr, dc in pipe_types[original_char]]
            return verify_symbol(mirrored_connections, local_grid_array, x_row, x_col), is_starting_pos
        else:
            # Try all possible combinations of connections
            for guess_char, mirrored_connections in pipe_types.items():
                mirrored_connections = [(-dr, -dc) for dr, dc in mirrored_connections]
                result = verify_symbol(mirrored_connections, local_grid_array, x_row, x_col)
                if result:
                    print(f"Found starting position! Assuming 'S' is {guess_char}")
                    is_starting_pos = True
                    return result, is_starting_pos
    if debug:
        print(f"Invalid pipe at {x_row}, {x_col}, when checking: {original_char}, found {valid_pipes} valid pipes")
    return None, is_starting_pos

def traverse_loop(start, pipes):
    """
    Iteratively traverse the loop starting from 'start', counting steps until returning to 'start'.
    Returns and all positions visited along the way.
    """
    current_pos = start
    prev_pos = None
    steps = 0
    visited = set() 
    while True:
        next_pos = None
        for neighbor in pipes[current_pos]:
            if neighbor != prev_pos or len(pipes[current_pos]) == 1:
                visited.add(neighbor)
                next_pos = neighbor
                break

        if next_pos == start and current_pos != start:
            # Completed the loop and returned to the starting position
            return steps, visited

        prev_pos = current_pos
        current_pos = next_pos
        steps += 1

def show_grid(valid_pipes):
    # Update the grid with the new pipes, take shape  and mark pipes with emojis
    new_grid = np.zeros(content.shape, dtype=str)
    new_grid.fill('🚫')
    for pos, neighbors in valid_pipes.items():
        new_grid[pos] = '🔗'
    display(new_grid)



with open('adventofcode.com_2023_day_10_input.txt', 'r') as f:
    input_string = f.read()

content = np.array(split_input(input_string))
display(content)
# Loop through every character, take a 3x3 grid around it, and check if it's a valid pipe
valid_pipes = {}
for row in range(content.shape[0]):
    for col in range(content.shape[1]):
        debug = False
        # Handle edge cases by creating partial grids
        grid_slice = content[max(0, row-1):min(content.shape[0], row+2), max(0, col-1):min(content.shape[1], col+2)].copy()
        # Determine the position of 'X' in the grid slice
        x_row, x_col = min(1, row), min(1, col)
        # Save the original character and mark the current position with 'X'
        original_char = grid_slice[x_row, x_col]
        if original_char == '-':
            debug = False
        grid_slice[x_row, x_col] = 'X'
        neighbors, found_start = check_pipe(x_row, x_col,grid_slice, original_char,debug)
        if neighbors:
            # print(f"Found valid pipe at {row}, {col}")
            valid_pipes[(row, col)] = [(row+dr, col+dc) for dr, dc in neighbors]
        if found_start:
            print(f"Setting starting position to {row}, {col}")
            starting_pos = (row, col)
# display(valid_pipes)
print(f"Found {len(valid_pipes)} valid pipes")
if starting_pos == (0,0):
    raise Exception("No starting position found!")
total_steps, visited = traverse_loop(starting_pos, valid_pipes)
print(f"Total steps in the loop: {total_steps}")
print(f"Half of the steps: {total_steps // 2 + 1}")

# Filter out the pipes that were not visited
valid_pipes = {k: v for k, v in valid_pipes.items() if k in visited}
show_grid(valid_pipes)

array([['.', '7', 'F', ..., '7', '.', 'F'],
       ['F', 'L', '7', ..., '.', 'F', '|'],
       ['7', '-', '7', ..., '-', 'F', '.'],
       ...,
       ['|', '7', '-', ..., 'L', '.', '.'],
       ['-', '|', 'F', ..., 'L', '-', '7'],
       ['L', 'J', 'J', ..., '|', 'J', 'J']], dtype='<U1')

Found starting position! Assuming 'S' is L
Setting starting position to 111, 14
Found 15039 valid pipes
Total steps in the loop: 13623
Half of the steps: 6812


array([['🚫', '🚫', '🚫', ..., '🚫', '🚫', '🚫'],
       ['🚫', '🚫', '🚫', ..., '🚫', '🚫', '🚫'],
       ['🚫', '🚫', '🚫', ..., '🚫', '🚫', '🚫'],
       ...,
       ['🚫', '🚫', '🚫', ..., '🚫', '🚫', '🚫'],
       ['🚫', '🚫', '🚫', ..., '🚫', '🚫', '🚫'],
       ['🚫', '🚫', '🚫', ..., '🚫', '🚫', '🚫']], dtype='<U1')

In [15]:
import numpy as np
from collections import deque

# Your existing functions and setup remain unchanged
def traverse_loop(start, pipes):
    """
    Iteratively traverse the loop starting from 'start', counting steps until returning to 'start'.
    Returns and all positions visited along the way.
    """
    current_pos = start
    prev_pos = None
    steps = 0
    visited = set() 
    while True:
        next_pos = None
        for neighbor in pipes[current_pos]:
            if neighbor != prev_pos or len(pipes[current_pos]) == 1:
                visited.add(neighbor)
                next_pos = neighbor
                break

        if next_pos == start and current_pos != start:
            # Completed the loop and returned to the starting position
            return steps, visited

        prev_pos = current_pos
        current_pos = next_pos
        steps += 1


# Traverse the loop and count the steps
total_steps, visited = traverse_loop(starting_pos, valid_pipes)
print(f"Total steps in the loop: {total_steps}")
print(f"Half of the steps: {total_steps // 2 + 1}")

# Filter out the pipes that were not visited
valid_pipes = {k: v for k, v in valid_pipes.items() if k in visited}


Total steps in the loop: 13623
Half of the steps: 6812
